In [3]:
import numpy as np
import joblib
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import string

# Define the MyBinaryRelevance class here or import it if it's defined in another module
class MyBinaryRelevance:
    def __init__(self, classifier, max_iterations=10):
        self.classifier = classifier
        self.models = []
        self.max_iterations = max_iterations
        self.history = {"loss": []} # Initialize history dictionary

    def fit(self, X_train, Y_train):
        num_labels = Y_train.shape[1]

        for i in range(num_labels):
            model = self.classifier(kernel='linear')
            model.fit(X_train, Y_train[:, i])
            self.models.append(model)

            if i + 1 == self.max_iterations:
                break

    def predict(self, X_test):
        predictions = np.zeros((X_test.shape[0], len(self.models)))

        for i, model in enumerate(self.models):
            predictions[:, i] = model.predict(X_test)

        return predictions

# Load the saved model
loaded_classifier = joblib.load("svm_model.joblib")

# Load the CountVectorizer used for training
count_vector = joblib.load("count_vector.joblib")

# Preprocess the new comment
new_comment = "you fucking stupid dwarf "

# Initialize the lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Remove punctuation and convert to lowercase
new_comment = new_comment.lower().translate(str.maketrans('', '', string.punctuation))

# Remove stop words and apply stemming and lemmatization
stop_words = set(stopwords.words('english'))
word_list = []
for word in new_comment.split():
    if word not in stop_words:
        word_list.append(stemmer.stem(lemmatizer.lemmatize(word, pos="v")))
new_comment = " ".join(word_list)

# Vectorize the new comment
new_comment_vectorized = count_vector.transform([new_comment]).toarray()

# Use the loaded model for prediction
predictions_svm = loaded_classifier.predict(new_comment_vectorized)

# Display the predictions
label_plot = ['toxic', 'obscene', 'insult']
for i, label in enumerate(label_plot):
    print(f"{label}: {predictions_svm[0, i]}")


toxic: 1.0
obscene: 1.0
insult: 1.0
